In [54]:
import matplotlib.pyplot as plt
import torch
import torchvision


from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchinfo import summary
import modules.data_setup as data_setup
from modules.engine import train_step,test_step,train
from modules.create_writer import create_writer
from modules.predict_plot import device
from modules.save_model import save_model
import os


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [55]:
version = torch.__version__
print(f"PyTorch Version: {version}")

PyTorch Version: 2.8.0


In [56]:
import requests
from pathlib import Path
import zipfile

data_path = Path('data')
image_path = data_path / 'pizza_steak_sushi_20_percent'

if image_path.is_dir():
    print(f'{image_path} exists')
else:
    print(f'{image_path} does not exist, creating...')
    image_path.mkdir(parents=True, exist_ok=True)

    with open(data_path / 'pizza_steak_sushi_20_percent.zip','wb') as f:
         request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
         print('Downloading pizza_steak_sushi_20_percent data')
         f.write(request.content)
    with zipfile.ZipFile(data_path / 'pizza_steak_sushi_20_percent.zip','r') as zip_ref:
        print("Extracting pizza_steak_sushi_20_percent.zip")
        zip_ref.extractall(image_path)

    os.remove(data_path / 'pizza_steak_sushi_20_percent.zip')

data/pizza_steak_sushi_20_percent exists


In [57]:
def set_seed(seed:int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [58]:
train_dir = image_path/'train'
test_dir = image_path/'test'
normalize = transforms.Normalize(
    mean=[0.485,0.456,0.406],
    std=[0.229,0.224,0.225]
)
simple_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])
print(simple_transform)


Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [59]:
train_dataloader,test_dataloader,class_names = data_setup.create_dataloaders(
                                                train_dir=train_dir,
                                                test_dir=test_dir,
                                                train_transform=simple_transform,
                                                test_transform=simple_transform,
                                                batch_size=32)
train_dataloader,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x1565af1d0>,
 ['pizza', 'steak', 'sushi'])

In [60]:
weights =  torchvision.models.EfficientNet_B1_Weights.DEFAULT
effnetb1 = torchvision.models.efficientnet_b1(weights=weights)
model_summary = summary(
    model=effnetb1,
    input_size=(32,3,224,224),
    verbose=0,
    col_names = ['input_size', 'output_size', 'num_params','trainable'],
    col_width=20,
    row_settings=['var_names']
)
model_summary

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [61]:
for param in effnetb1.features.parameters():
    param.requires_grad=False

set_seed()
effnetb1.classifier = nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,out_features=len(class_names),bias=True)
)

In [62]:
model_summary = summary(
    model=effnetb1,
    input_size=(32,3,224,224),
    verbose=0,
    col_names = ['input_size', 'output_size', 'num_params','trainable'],
    col_width=20,
    row_settings=['var_names']
)
model_summary

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [63]:
experiment_number = 0
set_seed()
model_name = "effnetb1"
num_epochs = [5,10]
for epochs in num_epochs:
    experiment_number += 1
    print(f"[INFO] Experiment number: {experiment_number}")
    print(f"[INFO] Model: {model_name}")
    print(f"[INFO] Number of epochs: {epochs}")

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=effnetb1.parameters(),lr=0.001)

    train(
    model = effnetb1,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=epochs,
    device=device,
    writer=create_writer(
        experiment_name="data_20_percent",
        model_name=model_name,
        extra=f"{epochs}_epochs"
    ))

    save_path = f'{model_name}_data_20_percent_{epochs}_epochs.pth'
    save_model(
    model=effnetb1,
    target_dir="models",
    model_name=save_path
    )
    print("-"*50 + "\n")


[INFO] Experiment number: 1
[INFO] Model: effnetb1
[INFO] Number of epochs: 5
[INFO] Created SummaryWriter. saving to runs/2025-10-01/data_20_percent/effnetb1/5_epochs...
Epoch 1| Train Loss: 1.028| Train Acc: 0.529| Test Loss: 0.880| Test Acc: 0.880
Epoch 2| Train Loss: 0.807| Train Acc: 0.850| Test Loss: 0.722| Test Acc: 0.914
Epoch 3| Train Loss: 0.721| Train Acc: 0.863| Test Loss: 0.615| Test Acc: 0.904
Epoch 4| Train Loss: 0.629| Train Acc: 0.869| Test Loss: 0.547| Test Acc: 0.929
Epoch 5| Train Loss: 0.580| Train Acc: 0.879| Test Loss: 0.469| Test Acc: 0.941
[INFO]Saving model to models/effnetb1_data_20_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Model: effnetb1
[INFO] Number of epochs: 10
[INFO] Created SummaryWriter. saving to runs/2025-10-01/data_20_percent/effnetb1/10_epochs...
Epoch 1| Train Loss: 0.517| Train Acc: 0.842| Test Loss: 0.405| Test Acc: 0.941
Epoch 2| Train Loss: 0.459| Train Acc: 0.885| Test Loss: 